# 📌 추천 시스템

## 콘텐츠 기반 필터링 (Content-Based Filtering)




## 영화 추천
### 데이터셋: TMDB 5000 (https://www.kaggle.com/tmdb/tmdb-movie-metadata)

### 콘텐츠 기반 필터링 구현 프로세스  

1. 콘텐츠에 대한 여러 텍스트 정보들을 피쳐 벡터화
2. 코사인 유사도로 콘텐츠별 유사도 계산
3. 콘텐츠 별로 가중 평점을 계산
4. 유사도가 높은 콘텐츠 중에 평점이 좋은 콘텐츠 순으로 추천

---

## 🎬 TMDB 5000 영화 데이터셋 설명

### TMDB_5000_credits.csv  

| 필드명      | 설명                                      |
|------------|-------------------------------------------|
| **movie_id** | 각 영화의 고유 식별자                     |
| **cast**    | 주연 및 조연 배우의 이름                   |
| **crew**    | 감독, 편집자, 작곡가, 작가 등의 스태프 이름 |

---

### TMDB_5000_movies.csv

| 필드명                  | 설명                                      |
|------------------------|------------------------------------------|
| **budget**             | 영화 제작 예산                           |
| **genres**              | 영화 장르 (액션, 코미디, 스릴러 등)     |
| **homepage**           | 영화 공식 홈페이지 링크                  |
| **id**                 | `movie_id`와 동일한 영화의 고유 식별자   |
| **keywords**           | 영화와 관련된 키워드 또는 태그           |
| **original_language**  | 영화의 원어                               |
| **original_title**     | 번역 또는 각색 이전의 원제               |
| **overview**           | 영화의 간략한 줄거리                     |
| **popularity**         | 영화의 인기도를 나타내는 수치            |
| **production_companies** | 영화 제작사                             |
| **production_countries** | 영화가 제작된 국가                     |
| **release_date**       | 영화 개봉 날짜                           |
| **revenue**            | 전 세계에서 벌어들인 수익                |
| **runtime**            | 영화의 총 상영 시간 (분 단위)            |
| **status**             | 영화의 상태 (`Released`, `Rumored`)      |
| **tagline**            | 영화의 대표 문구                         |
| **title**              | 영화 제목                                |
| **vote_average**       | 영화의 평균 평점                        |
| **vote_count**         | 영화가 받은 총 투표 수                   |

---

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

movies = pd.read_csv('./data/tmdb/tmdb_5000_movies.csv')
print(f"Data Frame shape: {movies.shape}")
movies.head(1)

Data Frame shape: (4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [2]:
## 중요 컬럼만 추출
col_li = [
            'id',           # 아이디
            'title',        # 영화 제목
            'genres',       # 영화 장르
            'vote_average', # 영화의 평균 평점
            'vote_count',   # 영화가 받은 총 투표 수  
            'popularity',   # 영화의 인기도를 나타내는 수치
            'keywords',     # 영화와 관련된 키워드 또는 태그
            'overview']     #영화의 간략한 줄거리

movies_df = movies[col_li]

- genres, keywords 등은 리스트 내에 여러 개의 사전으로 이루어진 형태로 입력되어 있음
- 각 장르, 키워드 명칭은 사전의 key인 name으로 추출 가능

In [3]:
# 컬럼 길이 늘려서 출력
pd.set_option('max_colwidth', 80)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id..."


In [4]:
from ast import literal_eval

# DataFrame의 슬라이스에서 값을 직접 수정할때 나오는 경고 -> 무시

# 문자열을 객체로 변경: 리스트 내의 사전
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [5]:
# 객제에서 name만 추출: 사전 마다 name을 추출
movies_df['genres'] = movies_df['genres'].apply(lambda x: [dic['name'] for dic in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [dic['name'] for dic in x])

In [6]:
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futu..."


장르 콘텐츠 필터링을 이용한 영화 추천, 장르 문자열을 count 백터화 후에 코사인 유사도로 각 영화를 비교  
장르 문자열의 Count기반 피쳐 벡터화

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환.
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))

# CountVectorizer
count_vect = CountVectorizer(min_df=1, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


장르에 따른 영화별 코사인 유사도 추출

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [9]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


특정 영화와 장르별 유사도가 높은 영화를 반환하는 함수 생성

In [10]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):

    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name값인 DataFrame 추출
    title_movie = df[df['title'] == title_name]

    # title_name을 가진 DataFrame의 index 갹체를 ndarray로 반환, sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    # 추출된 top_n index들 출력. top_n index는 2차원 데이터
    # DataFrame에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

In [11]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


평점이 높은 영화 정보 확인

In [12]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


평가 횟수에 대한 가중치 부여된 평점 (Weighted Rating) 계산


가중평균(Wighted Rating) = (v/(v+m)) * R +(m/(v+m)) * C


- v: 개별 영화에 평점을 투표한 횟수
- m : 평점을 부여하기 위한 최소 투표 횟수
- R : 개별 영화에 대한 평균 평점
- C : 전체 영화에 대한 평점

In [13]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:',round(C,3), 'm:',round(m,3))

C: 6.092 m: 370.2


In [14]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [15]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [16]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):

    # 입력한 영화의 index
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    
# 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

In [17]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
